In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import gurobipy as gp
import sys
from gurobipy import GRB
from itertools import combinations_with_replacement, product
from contextlib import redirect_stdout
import os
import numpy as np
from app.load import load_solution, load_sol_from_ampl_output
from app.utils import set_initial_solution, write_sol_xml
import json
from app.load import load_solution

In [ ]:
model = gp.read("lp_models/1_8.lp")
with open("Initial_sols/1_8.json", "r") as file:
    display_output = file.read()
    variables, obj = load_solution(
        "Early_8_comp_best.xml", []
    )  # json.loads(display_output)["Vars"]
    for var in variables:
        model.getVarByName(var).start = 1
# model.setParam("MIPFocus", 1)
model.setParam("TimeLimit", 5)
model.optimize()

In [ ]:
variable

In [ ]:
for variable in model.getVars():
    print(
        f"Variable {variable.VarName}: Reduced Cost = {variable.Obj}, Shadow Price = {variable.Pi}"
    )

In [ ]:
for lp_model in os.listdir("lp_models"):
    filename = lp_model[:-3]
    if not os.path.exists("results"):
        os.makedirs("results")
    if not os.path.exists(f"results/{filename}"):
        os.makedirs(f"results/{filename}")
    log_file = open(f"results/{filename}/logs.txt", "w")
    with redirect_stdout(log_file):
        model = gp.read(f"./lp_models/{filename}.lp")
        model.setParam("MIPFocus", 1)
        # model.setObjective(sum(model.getVarByName("dse1[{i},{j},0]") for i, j in product(range(4),range(4))))  # + model.getVarByName("dFA2[14,16,1]")
        model.setParam("TimeLimit", 10)
        model.optimize()
        model.write(f"results/{filename}/initial_sol.json")
    log_file.close()

In [ ]:
model = gp.read(f"./lp_models/1_8.lp")
model.setParam("MIPFocus", 1)
with open("results/1_8_/initial_sol.json", "r") as file:
    display_output = file.read()
    variables = json.loads(display_output)["Vars"]
    for var in variables:
        model.getVarByName(var["VarName"]).start = var["X"]
# model.setObjective(
# sum(model.getVarByName(f"dse1[{i},{j},0]") for i, j in product(range(4), range(4)))
# )  # + model.getVarByName("dFA2[14,16,1]")
model.setParam("TimeLimit", 100)
objec = model.getObjective()
# model2 = model.copy()
# model.setObjective(0.0)
model.optimize()

# model.write(f"results/testInstanceDemo/initial_sol1.json")

In [ ]:
all_teams = np.arange(12)
np.random.shuffle(all_teams)
subset_teams = np.array_split(all_teams, 3)
fix_teams = [np.setdiff1d(all_teams, subset_teams[i]) for i in range(3)]
fix_teams

In [ ]:
subset_teams

In [ ]:
import numpy as np

# Read the AMPL display output from the file
display_output_file = "solution_1_8.txt"
with open(display_output_file, "r") as file:
    display_output = file.read()


# Function to parse the table and create a 2D numpy array
def parse_table(table_str):
    lines = table_str.strip().split("\n")
    team2s = list(map(int, lines[0].split()[1:-1]))
    slots = [list(map(int, line.split()))[0] for line in lines[1:] if line != ";"]
    # Extract the first column (indices)
    values = [list(map(int, line.split()))[1:] for line in lines[1:] if line != ";"]
    return team2s, slots, np.array(values)


# Split the output into individual tables for each variable
variable_tables = display_output.strip().split("\n\n")

# Create a dictionary to store the 3D arrays
variables = {}
i = 0
# Parse each table and create 2D numpy arrays
for table in variable_tables:
    if table[0] == "_":
        continue
    elif table[0] in ["d", "h", "y"]:
        break
    lines = table.strip().split("\n")
    var_name = lines[1].split()[0]  # Extract the variable name
    team2s, slots, var_array = parse_table("\n".join(lines[1:]))
    for slot_index, slot in enumerate(slots):
        for team2_index, team2 in enumerate(team2s):
            variables[f"x[{i},{team2},{slot}]"] = var_array[slot_index, team2_index]
    i += 1

In [ ]:
for v in variables:
    model2.getVarByName(v).start = variables[v]

In [ ]:
json_file = open(f"results/Test1/initial_sol.json", "r")
sol_json = json.load(json_file)
sol_json["Vars"]

In [ ]:
for variable in model2.getVars():
    variable.start = model.getVarByName(variable.varName).x
model2.optimize()

In [ ]:
for i, j, k in [
    [0, 1, 0],
    [0, 2, 1],
    [0, 3, 5],
    [0, 4, 4],
    [0, 5, 7],
    [0, 6, 30],
    [0, 7, 18],
    [0, 8, 22],
    [0, 9, 31],
    [0, 10, 11],
    [0, 11, 15],
    [0, 12, 8],
    [0, 13, 9],
    [0, 14, 12],
    [0, 15, 13],
    [0, 16, 32],
    [0, 17, 24],
    [1, 0, 14],
    [1, 2, 2],
    [1, 3, 11],
    [1, 4, 1],
    [1, 5, 3],
    [1, 6, 5],
    [1, 7, 6],
    [1, 8, 28],
    [1, 9, 7],
    [1, 10, 10],
    [1, 11, 17],
    [1, 12, 23],
    [1, 13, 32],
    [1, 14, 13],
    [1, 15, 8],
    [1, 16, 22],
    [1, 17, 4],
    [2, 0, 16],
    [2, 1, 15],
    [2, 3, 0],
    [2, 4, 3],
    [2, 5, 10],
    [2, 6, 6],
    [2, 7, 5],
    [2, 8, 7],
    [2, 9, 4],
    [2, 10, 12],
    [2, 11, 24],
    [2, 12, 11],
    [2, 13, 13],
    [2, 14, 9],
    [2, 15, 17],
    [2, 16, 14],
    [2, 17, 19],
    [3, 0, 6],
    [3, 1, 12],
    [3, 2, 18],
    [3, 4, 2],
    [3, 5, 8],
    [3, 6, 23],
    [3, 7, 24],
    [3, 8, 10],
    [3, 9, 28],
    [3, 10, 22],
    [3, 11, 1],
    [3, 12, 3],
    [3, 13, 4],
    [3, 14, 7],
    [3, 15, 16],
    [3, 16, 15],
    [3, 17, 25],
    [4, 0, 17],
    [4, 1, 16],
    [4, 2, 8],
    [4, 3, 13],
    [4, 5, 0],
    [4, 6, 7],
    [4, 7, 11],
    [4, 8, 14],
    [4, 9, 5],
    [4, 10, 6],
    [4, 11, 12],
    [4, 12, 10],
    [4, 13, 15],
    [4, 14, 19],
    [4, 15, 21],
    [4, 16, 24],
    [4, 17, 27],
    [5, 0, 19],
    [5, 1, 9],
    [5, 2, 21],
    [5, 3, 20],
    [5, 4, 22],
    [5, 6, 4],
    [5, 7, 15],
    [5, 8, 6],
    [5, 9, 11],
    [5, 10, 17],
    [5, 11, 25],
    [5, 12, 12],
    [5, 13, 24],
    [5, 14, 26],
    [5, 15, 18],
    [5, 16, 1],
    [5, 17, 2],
    [6, 0, 29],
    [6, 1, 21],
    [6, 2, 22],
    [6, 3, 17],
    [6, 4, 18],
    [6, 5, 33],
    [6, 7, 10],
    [6, 8, 31],
    [6, 9, 8],
    [6, 10, 20],
    [6, 11, 14],
    [6, 12, 9],
    [6, 13, 11],
    [6, 14, 15],
    [6, 15, 24],
    [6, 16, 12],
    [6, 17, 3],
    [7, 0, 23],
    [7, 1, 20],
    [7, 2, 32],
    [7, 3, 31],
    [7, 4, 9],
    [7, 5, 14],
    [7, 6, 25],
    [7, 8, 3],
    [7, 9, 27],
    [7, 10, 16],
    [7, 11, 26],
    [7, 12, 13],
    [7, 13, 28],
    [7, 14, 4],
    [7, 15, 0],
    [7, 16, 17],
    [7, 17, 21],
    [8, 0, 25],
    [8, 1, 33],
    [8, 2, 26],
    [8, 3, 29],
    [8, 4, 32],
    [8, 5, 23],
    [8, 6, 13],
    [8, 7, 8],
    [8, 9, 9],
    [8, 10, 4],
    [8, 11, 5],
    [8, 12, 15],
    [8, 13, 21],
    [8, 14, 20],
    [8, 15, 1],
    [8, 16, 30],
    [8, 17, 11],
    [9, 0, 21],
    [9, 1, 25],
    [9, 2, 29],
    [9, 3, 30],
    [9, 4, 23],
    [9, 5, 32],
    [9, 6, 26],
    [9, 7, 19],
    [9, 8, 0],
    [9, 10, 24],
    [9, 11, 10],
    [9, 12, 20],
    [9, 13, 18],
    [9, 14, 17],
    [9, 15, 15],
    [9, 16, 3],
    [9, 17, 22],
    [10, 0, 3],
    [10, 1, 29],
    [10, 2, 33],
    [10, 3, 27],
    [10, 4, 28],
    [10, 5, 13],
    [10, 6, 1],
    [10, 7, 30],
    [10, 8, 2],
    [10, 9, 14],
    [10, 11, 8],
    [10, 12, 0],
    [10, 13, 23],
    [10, 14, 21],
    [10, 15, 31],
    [10, 16, 25],
    [10, 17, 15],
    [11, 0, 33],
    [11, 1, 19],
    [11, 2, 28],
    [11, 3, 32],
    [11, 4, 31],
    [11, 5, 30],
    [11, 6, 16],
    [11, 7, 7],
    [11, 8, 27],
    [11, 9, 6],
    [11, 10, 9],
    [11, 12, 2],
    [11, 13, 22],
    [11, 14, 23],
    [11, 15, 11],
    [11, 16, 18],
    [11, 17, 13],
    [12, 0, 26],
    [12, 1, 18],
    [12, 2, 27],
    [12, 3, 21],
    [12, 4, 30],
    [12, 5, 29],
    [12, 6, 32],
    [12, 7, 33],
    [12, 8, 24],
    [12, 9, 16],
    [12, 10, 19],
    [12, 11, 4],
    [12, 13, 14],
    [12, 14, 31],
    [12, 15, 22],
    [12, 16, 28],
    [12, 17, 17],
    [13, 0, 27],
    [13, 1, 26],
    [13, 2, 31],
    [13, 3, 19],
    [13, 4, 20],
    [13, 5, 5],
    [13, 6, 2],
    [13, 7, 12],
    [13, 8, 17],
    [13, 9, 1],
    [13, 10, 7],
    [13, 11, 0],
    [13, 12, 25],
    [13, 14, 33],
    [13, 15, 10],
    [13, 16, 8],
    [13, 17, 29],
    [14, 0, 28],
    [14, 1, 24],
    [14, 2, 30],
    [14, 3, 14],
    [14, 4, 25],
    [14, 5, 27],
    [14, 6, 0],
    [14, 7, 22],
    [14, 8, 16],
    [14, 9, 2],
    [14, 10, 18],
    [14, 11, 29],
    [14, 12, 1],
    [14, 13, 3],
    [14, 15, 6],
    [14, 16, 10],
    [14, 17, 8],
    [15, 0, 20],
    [15, 1, 27],
    [15, 2, 25],
    [15, 3, 26],
    [15, 4, 29],
    [15, 5, 28],
    [15, 6, 19],
    [15, 7, 2],
    [15, 8, 12],
    [15, 9, 33],
    [15, 10, 5],
    [15, 11, 3],
    [15, 12, 7],
    [15, 13, 30],
    [15, 14, 32],
    [15, 16, 23],
    [15, 17, 9],
    [16, 0, 2],
    [16, 1, 31],
    [16, 2, 20],
    [16, 3, 9],
    [16, 4, 33],
    [16, 5, 16],
    [16, 6, 27],
    [16, 7, 29],
    [16, 8, 19],
    [16, 9, 13],
    [16, 10, 26],
    [16, 11, 21],
    [16, 12, 5],
    [16, 13, 6],
    [16, 14, 11],
    [16, 15, 4],
    [16, 17, 0],
    [17, 0, 10],
    [17, 1, 30],
    [17, 2, 23],
    [17, 3, 33],
    [17, 4, 26],
    [17, 5, 31],
    [17, 6, 28],
    [17, 7, 1],
    [17, 8, 18],
    [17, 9, 12],
    [17, 10, 32],
    [17, 11, 20],
    [17, 12, 6],
    [17, 13, 16],
    [17, 14, 5],
    [17, 15, 14],
    [17, 16, 7],
]:
    model.getVarByName(f"x[{i},{j},{k}]").setAttr("Start", 1)

In [ ]:
model.setParam("TimeLimit", 100)
model.optimize()

In [ ]:
v = model.getVars()
v[1].Start

In [ ]:
# model.setObjective(objec)
model.update()
# model.getObjective()
# model2.setObjective()
# model2.update()
# model2.getObjective()

In [ ]:
v[0]

In [ ]:
import xml.etree.ElementTree as ET

In [ ]:
if model.status == gp.GRB.OPTIMAL:
    # Create the root element of the XML document
    root = ET.Element("Solution")

    # Create an element to store the objective value
    metadata_element = ET.SubElement(root, "MetaData")
    objective_element = ET.SubElement(metadata_element, "ObjectiveValue")
    objective_element.set("objective", str(model.objVal))

    # Create an element to store variable values
    games_element = ET.SubElement(root, "Games")
    for variable in model.getVars():
        if variable.x > 0 and variable.varName[0] == "x":
            variable_element = ET.SubElement(games_element, "ScheduledMatch")
            variable_element.set("home", variable.varName[2])
            variable_element.set("away", variable.varName[4])
            variable_element.set("slot", variable.varName[6])

    # Create the XML tree
    tree = ET.ElementTree(root)

    # Write the XML tree to a file
    tree.write("results.xml")

    print("Optimization results saved to results.xml.")
else:
    print("Optimization did not converge to an optimal solution.")

In [ ]:
filename = xml_file[:-4]
f = os.path.join(directory, f"{filename}.xml")
problem = load_problem(f)
solution = Solution(
    problem=problem,
    representative=load_solution(
        f"results/{filename}/initial_sol.xml", dummy_init_sol(problem)
    )[0],
)
solution

In [ ]:
# Load the AMPL model
for i, j in product(range(1, 4), range(1, 16)):
    model = gp.read(f"./lp_models/{i}_{j}.lp")
    model.setParam("MIPFocus", 1)
    # model.setObjective(sum(model.getVarByName("dse1[{i},{j},0]") for i, j in product(range(4),range(4))))  # + model.getVarByName("dFA2[14,16,1]")
    model.setParam("TimeLimit", 10)
    model.optimize()
    print("------------------")

In [ ]:
for i in range(len(v)):
    if v[i].X != 0:
        print(v[i].VarName, v[i].x)

In [ ]:
model.update()